In [ ]:
from utils import print_score
from preprocess import preprocess_fraud_data

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve, roc_auc_score, f1_score

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [38]:
ruta = "./data/dataset_balanceado.csv"
df = pd.read_csv(ruta)

data = preprocess_fraud_data(df)

X_train = data['X_train']
X_val = data['X_val']
X_test = data['X_test']
y_train = data['y_train']
y_val = data['y_val']
y_test = data['y_test']

**Modelo Base de Regresión Logística** 

In [39]:
modelo_base = LogisticRegression(solver='liblinear', random_state=42)
modelo_base.fit(X_train, y_train)

print_score(modelo_base, X_train, y_train, X_val, y_val, train=True)
print_score(modelo_base, X_train, y_train, X_val, y_val, train=False)

Train Result:
Accuracy Score: 90.91%
_______________________________________________
CLASSIFICATION REPORT:
                     0      1  accuracy    macro avg  weighted avg
precision     0.909091    0.0  0.909091     0.454545      0.826446
recall        1.000000    0.0  0.909091     0.500000      0.909091
f1-score      0.952381    0.0  0.909091     0.476190      0.865801
support    6000.000000  600.0  0.909091  6600.000000   6600.000000
_______________________________________________
Confusion Matrix: 
 [[6000    0]
 [ 600    0]]

Test Result:
Accuracy Score: 90.91%
_______________________________________________
CLASSIFICATION REPORT:
                     0      1  accuracy    macro avg  weighted avg
precision     0.909091    0.0  0.909091     0.454545      0.826446
recall        1.000000    0.0  0.909091     0.500000      0.909091
f1-score      0.952381    0.0  0.909091     0.476190      0.865801
support    2000.000000  200.0  0.909091  2200.000000   2200.000000
___________________

> - El modelo no es capaz de identificar las transacciones fraudulentas, al tener tan pocos datos de la clase minoritaria `Fraude`, el modelo simplemente predice todas las transacciones como legítimas, debido a  que la mayor cantidad de muestras en los datos que tiene para aprender son de transacciones legítimas, por lo cual decide clasificar el fraude como no fraude. 
> - El problema de desbalanceo de clases es tan notable que es necesario aplicar algunos métodos para tratar con clases desbalanceadas, además de que la métrica `accuracy` en estos casos es una métrica engañosa, pues devuelve **90.91%**, lo que podría parecer un resultado bastante bueno cuando en realidad el modelo no predice bien, porque lo que realmente importa es predecir correctamente las transacciones fraudulentas, no las no fraudulentas.
> - En lo adelante hay que prestar más atención a las métricas que tienen más en cuenta a los datos de la clase minoritaria como `F1-score`, `Recall`, `precisión`, `matriz de confusión` , `AUC-ROC`, `PR`.

**Técnicas de Muestreo:**

- **Oversampling** (Sobremuestreo): Consiste en añadir copias de la clase minoritaria para aumentar su peso total.

- **Undersampling** (Submuestreo): Se basa en eliminar muestras de la clase mayoritaria para intentar equilibrar el número de mustras en cada clase.

- **Generación de muestras sintéticas**: En este caso se utilizan algoritmos como `SMOTE` que es capaz de generar más muestras de la clase minoritaria a partir de las que ya se tienen.

- **Uso de pesos en la función de pérdida**: Una estrategia efectiva al entrenar modelos es modificar la función de pérdida asignando pesos mayores a la clase minoritaria, esto incentiva al modelo a prestar más atención a los ejemplos menos representados.

- También se puede utilizar un enfoque híbrido que combina undersampling con oversampling.

In [42]:
print("Distribución original:", np.bincount(y_train))

#Undersampling Aleatorio
undersampler = RandomUnderSampler(sampling_strategy=1, random_state=42)  # 1:1 ratio
X_under, y_under = undersampler.fit_resample(X_train, y_train)

print("Distribución después de undersampling:", np.bincount(y_under))

model_under = LogisticRegression(solver='liblinear', random_state=42)
model_under.fit(X_under, y_under)

print_score(model_under, X_under, y_under, X_val, y_val, train=True)
print_score(model_under, X_train, y_train, X_val, y_val, train=False)

Distribución original: [6000  600]
Distribución después de undersampling: [600 600]
Train Result:
Accuracy Score: 55.00%
_______________________________________________
CLASSIFICATION REPORT:
                    0           1  accuracy    macro avg  weighted avg
precision    0.549834    0.550167      0.55     0.550001      0.550001
recall       0.551667    0.548333      0.55     0.550000      0.550000
f1-score     0.550749    0.549249      0.55     0.549999      0.549999
support    600.000000  600.000000      0.55  1200.000000   1200.000000
_______________________________________________
Confusion Matrix: 
 [[331 269]
 [271 329]]

Test Result:
Accuracy Score: 48.59%
_______________________________________________
CLASSIFICATION REPORT:
                     0           1  accuracy    macro avg  weighted avg
precision     0.917387    0.098361  0.485909     0.507874      0.842930
recall        0.477500    0.570000  0.485909     0.523750      0.485909
f1-score      0.628083    0.167770  0.

> Al reducir el número de muestras de la clase mayoritaria el modelo es capaz de detectar el fraude, aunque sigue clasificando muchos fraudes como no fraude, además clasifica mal demasiados falsos positivos, está detectando la mayoria como si fuese fraude

In [43]:
print("Distribución original:", np.bincount(y_train))

#Oversampling Aleatorio
oversampler = RandomOverSampler(sampling_strategy=1, random_state=42)
X_over, y_over = oversampler.fit_resample(X_train, y_train)

print("Distribución después de undersampling:", np.bincount(y_over))

model_over = LogisticRegression(solver='liblinear', random_state=42)
model_over.fit(X_over, y_over)

print_score(model_over, X_over, y_over, X_val, y_val, train=True)
print_score(model_over, X_train, y_train, X_val, y_val, train=False)

Distribución original: [6000  600]
Distribución después de undersampling: [6000 6000]
Train Result:
Accuracy Score: 53.37%
_______________________________________________
CLASSIFICATION REPORT:
                     0            1  accuracy     macro avg  weighted avg
precision     0.533937     0.533565   0.53375      0.533751      0.533751
recall        0.531000     0.536500   0.53375      0.533750      0.533750
f1-score      0.532464     0.535029   0.53375      0.533746      0.533746
support    6000.000000  6000.000000   0.53375  12000.000000  12000.000000
_______________________________________________
Confusion Matrix: 
 [[3186 2814]
 [2781 3219]]

Test Result:
Accuracy Score: 50.32%
_______________________________________________
CLASSIFICATION REPORT:
                     0           1  accuracy    macro avg  weighted avg
precision     0.907457    0.089236  0.503182     0.498347      0.833074
recall        0.505000    0.485000  0.503182     0.495000      0.503182
f1-score      0.6

In [44]:
#SMOTE (Generar muestras sintéticas)
smote = SMOTE(sampling_strategy=1, random_state=42)
X_smote, y_smote = smote.fit_resample(X_train, y_train)

# Entrenar modelo
model_smote = LogisticRegression(solver='liblinear', random_state=42)
model_smote.fit(X_smote, y_smote)

print_score(model_smote, X_smote, y_smote, X_val, y_val, train=True)
print_score(model_smote, X_train, y_train, X_val, y_val, train=False)

Train Result:
Accuracy Score: 63.04%
_______________________________________________
CLASSIFICATION REPORT:
                     0            1  accuracy     macro avg  weighted avg
precision     0.616983     0.647336  0.630417      0.632159      0.632159
recall        0.687833     0.573000  0.630417      0.630417      0.630417
f1-score      0.650485     0.607904  0.630417      0.629194      0.629194
support    6000.000000  6000.000000  0.630417  12000.000000  12000.000000
_______________________________________________
Confusion Matrix: 
 [[4127 1873]
 [2562 3438]]

Test Result:
Accuracy Score: 64.14%
_______________________________________________
CLASSIFICATION REPORT:
                     0           1  accuracy    macro avg  weighted avg
precision     0.908294    0.089261  0.641364     0.498777      0.833836
recall        0.673500    0.320000  0.641364     0.496750      0.641364
f1-score      0.773471    0.139586  0.641364     0.456528      0.715845
support    2000.000000  200.000

In [46]:
model_balanced = LogisticRegression(
    solver='liblinear',
    class_weight='balanced',  # Ajusta pesos automáticamente
    random_state=42
)

model_balanced.fit(X_train, y_train) 

print_score(model_balanced, X_train, y_train, X_val, y_val, train=True)
print_score(model_balanced, X_train, y_train, X_val, y_val, train=False)

Train Result:
Accuracy Score: 52.14%
_______________________________________________
CLASSIFICATION REPORT:
                     0           1  accuracy    macro avg  weighted avg
precision     0.918409    0.100780  0.521364     0.509595      0.844079
recall        0.519667    0.538333  0.521364     0.529000      0.521364
f1-score      0.663757    0.169777  0.521364     0.416767      0.618850
support    6000.000000  600.000000  0.521364  6600.000000   6600.000000
_______________________________________________
Confusion Matrix: 
 [[3118 2882]
 [ 277  323]]

Test Result:
Accuracy Score: 50.50%
_______________________________________________
CLASSIFICATION REPORT:
                     0           1  accuracy    macro avg  weighted avg
precision     0.909991    0.091827     0.505     0.500909      0.835612
recall        0.505500    0.500000     0.505     0.502750      0.505000
f1-score      0.649952    0.155159     0.505     0.402555      0.604971
support    2000.000000  200.000000     0.

**SMOTETomek:** Técnica híbrida que combina oversampling (SMOTE) con undersampling (Tomek Links).

El proceso es el siguiente:

- **Oversampling** (SMOTE): Primero, se aplica SMOTE al conjunto de datos de entrenamiento, esto crea nuevas muestras sintéticas de la clase minoritaria (fraude) hasta alcanzar un cierto balance (generalmente 1:1, pero se puede configurar).

- **Undersampling** (Tomek Links): Después de generar los nuevos puntos de fraude, se aplica Tomek Links, un "Tomek Link" es un par de puntos de clases opuestas que son los vecinos más cercanos el uno del otro, en este paso, se eliminan las muestras de la clase mayoritaria que forman parte de un Tomek Link.

SMOTE puede generar puntos sintéticos en regiones `ruidosas`, muy cerca de la frontera de decisión o incluso dentro del territorio de la clase mayoritaria. Al aplicar Tomek Links después, se `limpia` la frontera de decisión, eliminando puntos de la clase mayoritaria que están demasiado cerca de los puntos de la clase minoritaria (tanto los originales como los sintéticos). Esto ayuda a que el modelo aprenda una separación entre clases más clara y a menudo mejora la precisión sin sacrificar mucho el recall.

In [52]:
from imblearn.combine import SMOTETomek

# 1. Crear el objeto SMOTETomek
# Por defecto, SMOTE intentará balancear a 1:1
smt_tomek = SMOTETomek(random_state=42)

#Aplicar el remuestreo SOLO al conjunto de entrenamiento
print("Distribución original en entrenamiento:", np.bincount(y_train))
X_train_resampled, y_train_resampled = smt_tomek.fit_resample(X_train, y_train)
print("Distribución después de SMOTETomek:", np.bincount(y_train_resampled))

model_smotetomek = LogisticRegression(solver='liblinear', random_state=42)
model_smotetomek.fit(X_train_resampled, y_train_resampled)

print_score(model_smotetomek, X_train_resampled, y_train_resampled, X_val, y_val, train=True)
print_score(model_smotetomek, X_train, y_train, X_val, y_val, train=False)


Distribución original en entrenamiento: [6000  600]
Distribución después de SMOTETomek: [5982 5982]
Train Result:
Accuracy Score: 63.08%
_______________________________________________
CLASSIFICATION REPORT:
                     0            1  accuracy     macro avg  weighted avg
precision     0.617158     0.648061  0.630809      0.632609      0.632609
recall        0.689067     0.572551  0.630809      0.630809      0.630809
f1-score      0.651133     0.607970  0.630809      0.629552      0.629552
support    5982.000000  5982.000000  0.630809  11964.000000  11964.000000
_______________________________________________
Confusion Matrix: 
 [[4122 1860]
 [2557 3425]]

Test Result:
Accuracy Score: 64.14%
_______________________________________________
CLASSIFICATION REPORT:
                     0           1  accuracy    macro avg  weighted avg
precision     0.908294    0.089261  0.641364     0.498777      0.833836
recall        0.673500    0.320000  0.641364     0.496750      0.641364
f1-